# The Independent Agent Model

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def normalizeGame(somegame):
    means =  np.sum(somegame, 0)/4
#     print(means)
#     ranges = np.ptp(somegame)
    normalgame = (somegame - means)#/ranges
    return normalgame

def printGame(somegame):
    print(somegame[0], somegame[1],"\n",somegame[2],somegame[3])

In [3]:
import numpy as np

def softmax(x):
    e_x = np.exp(x - np.max(x))
    out = e_x / e_x.sum()
    return out

# Calculate probability of ending up in each cell

In [4]:
def gameProbabilities(somegame, wAa, wAb, wBa, wBb, gAb, gBa, ChoiceA, ChoiceB):
    ''' Calculate probability of ending up in each cell of a game'''  
    # Each player chooses one of the 4 cells to end up in
    # Their results are then averaged or multiplied (then normalized if necessary)
    game = normalizeGame(somegame)
    
    maxa,maxb = np.max(somegame,0)
    pa = np.array([0,0,0,0])
    pb = np.array([0,0,0,0])

    for i in range(4):
        pa[i] = game[i][0] * wAa + game[i][1] * gAb * wAb
        
    for i in range(4):
        pb[i] = game[i][0] * wBa * gBa + game[i][1]  * wBb

    pa = softmax(pa)
    pb = softmax(pb)
    
    probs = abs(np.multiply(pa, pb))
    scale = np.sum(probs)
    i = ChoiceA * 2  + ChoiceB
    
    return abs(pa[i] * pb[i] / scale)

# Probability of friends

$ P(friends | C_A, C_B, g) = \frac{P(C_A | friends) * P(friends)}{P(C_A, C_B)} $  
$ \propto P(C_A, C_B \space|\space friends) * P(friends) $  
$ \propto P(C_A, C_B \space|\space friends) $  
$ = P(C_A, C_B \space|\space W_A < W_B, \gamma = 1, ...)$  
$ \propto \frac{1}{n} \sum_{Wa, Wb}^{n} P(C_A, C_B \space|\space W_A < W_B, \gamma = 1, ...)$

In [5]:
def friendProb(game, choiceA, choiceB):
    gamma = 1
    wAa = 0
    wAb = 1

    wBa = 1
    wBb = 0
    prob = gameProbabilities(game, wAa, wAb, wBa, wBb, gamma, gamma, choiceA, choiceB)
#     print(probs)

#     return 1
    return prob


In [6]:
def enemyProb(game, choiceA, choiceB):

    gamma = -1
    wAa = 0
    wAb = 1

    wBa = 1
    wBb = 0


    prob = gameProbabilities(game, wAa, wAb, wBa, wBb, gamma, gamma, choiceA, choiceB)

    return prob


In [7]:
def strangerProb(game, choiceA, choiceB):
    probs = np.zeros([51, 51])
    wA = np.linspace(0.5,1,50)
    wB = np.linspace(0.5,1,50)
    gamma = 1
    for wAa in wA:
        wAb = 1 - wAa
        for wBb in wB:
            wBa = 1 - wBb
            prob = gameProbabilities(game, wAa, wAb, wBa, wBb, gamma, gamma, choiceA, choiceB)
            probs[int(wAa*100)- 50,int(wBb*100) - 50] = prob
#             print(prob, wAa, wAb, wBa, wBb, gamma)
#     print(probs)
    return np.sum(probs)/(51.*51.)


In [8]:
def FSE(somegame, choiceA, choiceB):
    F = friendProb(somegame,choiceA,choiceB)
    S = strangerProb(somegame,choiceA,choiceB)
    E = enemyProb(somegame,choiceA,choiceB)
#     print(F,S,E)
    fse = F+S+E
    return F/fse, S/fse, E/fse

In [9]:
pd = {"gameString":"pd", "game":[[8, 8],[0, 12],[12, 0],[4,4]], "choices":[0 ,1, 3]}                            # 0 1 3
threat = {"gameString":"threat", "game":[[12, 6],[6, 12],[6, 0],[0,6]], "choices":[ 0, 1, 2, 3]}                      #  0 1 2 3
disjunctive = {"gameString":"disjunctive", "game":[[12,12], [12,12], [12,12], [0,0]], "choices":[0, 3]}      #  0 3
coordination = {"gameString":"coordination", "game":[[12,12], [0,0], [0,0], [12,12]], "choices":[ 0, 1]}       #  0 1
singleControl = {"gameString":"singleControl", "game":[[6,6], [0,6], [6,6], [0,6]], "choices":[0, 1]}              #  0 1
games = [pd, threat, disjunctive, coordination, singleControl]

In [11]:
choicemap = {0:[0,0], 1:[0,1], 2:[1,0], 3:[1,1]}
for game in games:
    for choice in game["choices"]:
        probs = FSE(game["game"],choicemap[choice][0],choicemap[choice][1])
        print("{} \t {} \t FSE: {:f} {:f} {:f}".format(game["gameString"], choicemap[choice], *probs))

pd 	 [0, 0] 	 FSE: 0.556953 0.442860 0.000187
pd 	 [0, 1] 	 FSE: 0.133424 0.733153 0.133424
pd 	 [1, 1] 	 FSE: 0.000335 0.000267 0.999398
threat 	 [0, 0] 	 FSE: 0.509908 0.490089 0.000003
threat 	 [0, 1] 	 FSE: 0.509908 0.490089 0.000003
threat 	 [1, 0] 	 FSE: 0.000006 0.000041 0.999952
threat 	 [1, 1] 	 FSE: 0.000006 0.000041 0.999952
disjunctive 	 [0, 0] 	 FSE: 0.509900 0.490100 0.000000
disjunctive 	 [1, 1] 	 FSE: 0.000000 0.000000 1.000000
coordination 	 [0, 0] 	 FSE: 0.509900 0.490100 0.000000
coordination 	 [0, 1] 	 FSE: 0.000000 0.000000 1.000000
singleControl 	 [0, 0] 	 FSE: 0.518033 0.480683 0.001284
singleControl 	 [0, 1] 	 FSE: 0.002388 0.034341 0.963272
